In [11]:
!pip install vizdoom

In [11]:
# !cd github & git clone https://github.com/mwydmuch/ViZDoom

The system cannot find the path specified.
Cloning into 'ViZDoom'...
Updating files:   8% (174/2062)
Updating files:   9% (186/2062)
Updating files:  10% (207/2062)
Updating files:  11% (227/2062)
Updating files:  12% (248/2062)
Updating files:  13% (269/2062)
Updating files:  14% (289/2062)
Updating files:  15% (310/2062)
Updating files:  16% (330/2062)
Updating files:  17% (351/2062)
Updating files:  18% (372/2062)
Updating files:  19% (392/2062)
Updating files:  20% (413/2062)
Updating files:  21% (434/2062)
Updating files:  22% (454/2062)
Updating files:  23% (475/2062)
Updating files:  24% (495/2062)
Updating files:  25% (516/2062)
Updating files:  26% (537/2062)
Updating files:  27% (557/2062)
Updating files:  28% (578/2062)
Updating files:  29% (598/2062)
Updating files:  30% (619/2062)
Updating files:  31% (640/2062)
Updating files:  32% (660/2062)
Updating files:  33% (681/2062)
Updating files:  34% (702/2062)
Updating files:  35% (722/2062)
Updating files:  36% (743/2062)
Upd

In [1]:

# Import vizdoom for game env
from vizdoom import * 
# Import random for action sampling
import random
# Import time for sleeping
import time 
# Import numpy for identity matrix
import numpy as np

In [2]:
# Setup game
game = DoomGame()
game.load_config(r'C:\Users\indug\doom\ViZDoom\scenarios\deadly_corridor_s1.cfg')
game.init()

In [3]:
# This is the set of actions we can take in the environment
actions = np.identity(7, dtype=np.uint8)

In [4]:
state = game.get_state()

In [5]:
state.game_variables


array([100.,   0.,   0.,  -1.])

In [6]:
# Loop through episodes 
episodes = 10 
for episode in range(episodes): 
    # Create a new episode or game 
    game.new_episode()
    # Check the game isn't done 
    while not game.is_episode_finished(): 
        # Get the game state 
        state = game.get_state()
        # Get the game image 
        img = state.screen_buffer
        # Get the game variables - ammo
        info = state.game_variables
        # Take an action
        reward = game.make_action(random.choice(actions),4)
        # Print rewward 
        print('reward:', reward) 
        time.sleep(0.02)
    print('Result:', game.get_total_reward())
    time.sleep(2)

reward: 0.0
reward: 0.0
reward: 7.1137542724609375
reward: 8.533950805664062
reward: 3.7769317626953125
reward: 8.105987548828125
reward: 9.203216552734375
reward: 13.321395874023438
reward: 12.7210693359375
reward: 5.0521392822265625
reward: 1.044891357421875
reward: -3.2614593505859375
reward: -12.4375
reward: -11.914398193359375
reward: -11.06475830078125
reward: -12.229812622070312
reward: -8.249313354492188
reward: -5.564422607421875
reward: -9.046905517578125
reward: -7.876251220703125
reward: -3.1651611328125
reward: 7.0593719482421875
reward: 8.468719482421875
reward: 3.0680389404296875
reward: 0.145294189453125
reward: 0.0978546142578125
reward: 0.0659332275390625
reward: 0.0345611572265625
reward: 0.0
reward: 3.5823974609375
reward: -1.7559661865234375
reward: -7.946044921875
reward: -1.18780517578125
reward: 2.3775634765625
reward: 1.6036224365234375
reward: 1.0815582275390625
reward: 2.184722900390625
reward: 8.394332885742188
reward: 9.311203002929688
reward: 13.2294464111

In [7]:
game.close()

In [138]:
!pip install gym

In [8]:
# Import environment base class from OpenAI Gym
from gym import Env
# Import gym spaces 
from gym.spaces import Discrete, Box
# Import opencv 
import cv2

In [11]:
# game.get_state().screen_buffer.shape

In [10]:
# Create Vizdoom OpenAI Gym Environment
class VizDoomGym(Env): 
    # Function that is called when we start the env
    def __init__(self, render=False, config= r'C:\Users\indug\doom\ViZDoom\scenarios\deadly_corridor_s1.cfg'): 
        # Inherit from Env
        super().__init__()
        # Setup the game 
        self.game = DoomGame()
        self.game.load_config(config)
        
        # Render frame logic
        if render == False: 
            self.game.set_window_visible(False)
        else:
            self.game.set_window_visible(True)
        
        # Start the game 
        self.game.init()
        
        # Create the action space and observation space
        self.observation_space = Box(low=0, high=255, shape=(100,160,1), dtype=np.uint8) 
        self.action_space = Discrete(7)
        
        # Game variables: HEALTH DAMAGE_TAKEN HITCOUNT SELECTED_WEAPON_AMMO
        self.damage_taken = 0
        self.hitcount = 0
        self.ammo = 52 ## CHANGED
        
        
    # This is how we take a step in the environment
    def step(self, action):
        # Specify action and take step 
        actions = np.identity(7)
        movement_reward = self.game.make_action(actions[action], 4) 
        
        reward = 0 
        # Get all the other stuff we need to retun 
        if self.game.get_state(): 
            state = self.game.get_state().screen_buffer
            state = self.grayscale(state)
            
            # Reward shaping
            game_variables = self.game.get_state().game_variables
            health, damage_taken, hitcount, ammo = game_variables
            
            # Calculate reward deltas
            damage_taken_delta = -damage_taken + self.damage_taken
            self.damage_taken = damage_taken
            hitcount_delta = hitcount - self.hitcount
            self.hitcount = hitcount
            ammo_delta = ammo - self.ammo
            self.ammo = ammo
            
            reward = movement_reward + damage_taken_delta*10 + hitcount_delta*200  + ammo_delta*5 
            info = ammo
        else: 
            state = np.zeros(self.observation_space.shape)
            info = 0 
        
        info = {"info":info}
        done = self.game.is_episode_finished()
        
        return state, reward, done, info 
    
    # Define how to render the game or environment 
    def render(): 
        pass
    
    # What happens when we start a new game 
    def reset(self): 
        self.game.new_episode()
        state = self.game.get_state().screen_buffer
        return self.grayscale(state)
    
    # Grayscale the game frame and resize it 
    def grayscale(self, observation):
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (160,100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (100,160,1))
        return state
    
    # Call to close down the game
    def close(self): 
        self.game.close()

In [11]:
env = VizDoomGym(render=True)

In [12]:
env.step(3)

(array([[[32],
         [33],
         [25],
         ...,
         [27],
         [23],
         [24]],
 
        [[27],
         [33],
         [23],
         ...,
         [24],
         [24],
         [24]],
 
        [[20],
         [35],
         [23],
         ...,
         [24],
         [24],
         [24]],
 
        ...,
 
        [[75],
         [63],
         [62],
         ...,
         [44],
         [71],
         [60]],
 
        [[15],
         [48],
         [47],
         ...,
         [49],
         [69],
         [47]],
 
        [[22],
         [14],
         [26],
         ...,
         [57],
         [37],
         [39]]], dtype=uint8),
 0.0,
 False,
 {'info': 52.0})

In [13]:
state = env.reset()

In [14]:
env.reset()

array([[[32],
        [33],
        [25],
        ...,
        [27],
        [23],
        [24]],

       [[27],
        [33],
        [23],
        ...,
        [24],
        [24],
        [24]],

       [[20],
        [35],
        [23],
        ...,
        [24],
        [24],
        [24]],

       ...,

       [[75],
        [63],
        [62],
        ...,
        [44],
        [71],
        [60]],

       [[15],
        [48],
        [47],
        ...,
        [49],
        [69],
        [47]],

       [[22],
        [14],
        [26],
        ...,
        [57],
        [37],
        [39]]], dtype=uint8)

In [15]:
env.close()

In [16]:
# Import Environment checker
from stable_baselines3.common import env_checker

In [148]:
# env_checker.check_env(env)

In [149]:

# !pip install matplotlib

In [150]:
# from matplotlib import pyplot as plt

In [151]:

# plt.imshow(cv2.cvtColor(state, cv2.COLOR_BGR2RGB))

In [17]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [45]:
!pip install stable-baselines3[extra]


In [17]:
# Import os for file nav
import os 
# Import callback class from sb3
from stable_baselines3.common.callbacks import BaseCallback

In [48]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [55]:
CHECKPOINT_DIR = './train/train_corridor'
LOG_DIR = './logs/log_corridors1'

In [56]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [57]:

# import ppo for training
from stable_baselines3 import PPO

In [58]:
# Non rendered environment
env = VizDoomGym(config = r'C:\Users\indug\doom\ViZDoom\scenarios\deadly_corridor_s2.cfg')

In [46]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, learning_rate=0.00001, n_steps=8192, clip_range=.1, gamma=.95, gae_lambda=.9)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [31]:
model.learn(total_timesteps=400000, callback=callback)

Logging to ./logs/log_corridors1\PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 201      |
|    ep_rew_mean     | 31       |
| time/              |          |
|    fps             | 19       |
|    iterations      | 1        |
|    time_elapsed    | 107      |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 200        |
|    ep_rew_mean          | 83.1       |
| time/                   |            |
|    fps                  | 15         |
|    iterations           | 2          |
|    time_elapsed         | 262        |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01006293 |
|    clip_fraction        | 0.187      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.93      |
|    explained_variance   | 1.38e-05   |
|    l

In [32]:
# Import eval policy to test agent
from stable_baselines3.common.evaluation import evaluate_policy

In [39]:

# Reload model from disc
model = PPO.load('./train/train_corridor/best_model_60000')

In [40]:

# Create rendered environment
env = VizDoomGym(render=True)

In [42]:
# Evaluate mean reward for 10 games
mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=4)

In [123]:

mean_reward

-3.4269256591796875

In [44]:

for episode in range(10): 
    obs = env.reset()
    done = False
    total_reward = 0
    while not done: 
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        time.sleep(0.80)
        total_reward += reward
    print('Total Reward for episode {} is {}'.format(episode, total_reward))
    time.sleep(2)

Total Reward for episode 0 is 730.9943542480469
Total Reward for episode 1 is 1585.7324676513672
Total Reward for episode 2 is 1291.2153778076172
Total Reward for episode 3 is 1233.3893127441406
Total Reward for episode 4 is 786.0104370117188
Total Reward for episode 5 is 637.8748626708984
Total Reward for episode 6 is 1482.8897552490234
Total Reward for episode 7 is 735.5814971923828
Total Reward for episode 8 is 1661.4496002197266
Total Reward for episode 9 is 1370.3328704833984


In [45]:
env.close()

In [205]:
env = VizDoomGym(render=True)

In [214]:
env.step(1)

(array([[[ 6],
         [10],
         [ 9],
         ...,
         [ 6],
         [11],
         [ 4]],
 
        [[ 9],
         [ 9],
         [ 6],
         ...,
         [ 6],
         [ 6],
         [ 4]],
 
        [[10],
         [ 7],
         [ 7],
         ...,
         [ 6],
         [ 6],
         [ 9]],
 
        ...,
 
        [[75],
         [63],
         [62],
         ...,
         [44],
         [71],
         [60]],
 
        [[15],
         [48],
         [47],
         ...,
         [49],
         [69],
         [47]],
 
        [[22],
         [14],
         [26],
         ...,
         [57],
         [37],
         [39]]], dtype=uint8),
 0.0,
 False,
 {'info': 26.0})

In [204]:
env.close()

In [ ]:
model = PPO.load('./train/train_corridor/best_model_60000')

In [60]:
env = VizDoomGym(config = r'C:\Users\indug\doom\ViZDoom\scenarios\deadly_corridor_s2.cfg')
model.set_env(env)
model.learn(total_timesteps=560000, callback=callback)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Logging to ./logs/log_corridors1\PPO_6
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 82.1     |
|    ep_rew_mean     | 42.9     |
| time/              |          |
|    fps             | 29       |
|    iterations      | 1        |
|    time_elapsed    | 277      |
|    total_timesteps | 8192     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 92.3         |
|    ep_rew_mean          | 76.5         |
| time/                   |              |
|    fps                  | 43           |
|    iterations           | 2            |
|    time_elapsed         | 374          |
|    total_timesteps      | 16384        |
| train/                  |              |
|    approx_kl            | 0.0024113785 |
|    clip_fraction        | 0.

In [62]:

model = PPO.load('./train/train_corridor/best_model_440000')

In [63]:

# Create rendered environment
env = VizDoomGym(render=True)

In [64]:
# Evaluate mean reward for 10 games
mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=5)

In [65]:
mean_reward

-215.9957061767578

In [66]:

for episode in range(10): 
    obs = env.reset()
    done = False
    total_reward = 0
    while not done: 
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        time.sleep(0.80)
        total_reward += reward
    print('Total Reward for episode {} is {}'.format(episode, total_reward))
    time.sleep(2)

Total Reward for episode 0 is 1721.8127899169922
Total Reward for episode 1 is -111.82392883300781
Total Reward for episode 2 is 1161.8428497314453
Total Reward for episode 3 is 688.308349609375
Total Reward for episode 4 is 511.5319519042969
Total Reward for episode 5 is 802.8285064697266
Total Reward for episode 6 is 1125.2847137451172
Total Reward for episode 7 is 235.8668670654297
Total Reward for episode 8 is 644.769775390625
Total Reward for episode 9 is 463.5858154296875
